## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [6]:
import pandas as pd

In [9]:
df = pd.read_csv('../Data/data_statistics/cookie_cats.csv')

In [11]:
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [15]:
session_counts = df['userid'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'Є {multi_users} користувачів, які зустрічаються кілька разів у наборі даних.')

Є 0 користувачів, які зустрічаються кілька разів у наборі даних.


In [19]:
average_retention_7 = df.groupby('version')['retention_7'].mean()

In [21]:
print(average_retention_7)

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


gate_30 має середнє утримання на рівні 0.190201 (або близько 19%).

Це означає, що 19% користувачів, які грали у версію гри з параметром gate_30, повернулися в гру через 7 днів після встановлення.

gate_40 має середнє утримання на рівні 0.182000 (або близько 18%).

Це означає, що 18% користувачів, які грали у версію гри з параметром gate_40, повернулися в гру через 7 днів після встановлення.

Гіпотеза: версія gate_30 дає краще утримання через 7 днів після встановлення на 1%


2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [36]:
import numpy as np
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [103]:
df_gate_30 = df[df['version'] == 'gate_30']
df_gate_40 = df[df['version'] == 'gate_40']

n_gate_30 = df_gate_30['retention_7'].count()
n_gate_40 = df_gate_40['retention_7'].count()

successes_gate_30 = df_gate_30['retention_7'].sum()
successes_gate_40 = df_gate_40['retention_7'].sum()

successes = [successes_gate_30, successes_gate_40]
nobs = [n_gate_30, n_gate_40]

z_stat, pval = proportions_ztest(successes, nobs=nobs)

(lower_gate_30, upper_gate_30), (lower_gate_40, upper_gate_40) = proportion_confint(successes, nobs, alpha=0.05)

# не розумію, чому без наступної частини коду в мене в результатах плутаються вверхня і нижня межа?
if lower_gate_30 > upper_gate_30:
    lower_gate_30, upper_gate_30 = upper_gate_30, lower_gate_30

if lower_gate_40 > upper_gate_40:
    lower_gate_40, upper_gate_40 = upper_gate_40, lower_gate_40

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи gate_30: [{lower_gate_30:.3f}, {upper_gate_30:.3f}]')
print(f'Довірчий інтервал 95% для групи gate_40: [{lower_gate_40:.3f}, {upper_gate_40:.3f}]')

z statistic: 3.16
p-value: 0.002
Довірчий інтервал 95% для групи gate_30: [0.178, 0.187]
Довірчий інтервал 95% для групи gate_40: [0.186, 0.194]


1. p-значення (0.002) значно менше 0.05. Це означає, що ми можемо відхилити нульову гіпотезу (яка стверджує, що немає різниці між утриманням користувачів у двох групах). 

Так, є статистично значуща різниця між поведінкою користувачів у різних версіях гри.

2.  Ці два довірчі інтервали частково перекриваються, оскільки нижня межа групи gate_40 (0.186) знаходиться вище верхньої межі групи gate_30 (0.187) лише незначно.
   - Це означає, що, хоча й існує деяка ймовірність того, що середні показники утримання в цих групах можуть бути схожими, загалом інтервали вказують на те, що група gate_40 має вищий рівень утримання. Ці показники ще раз підтверджують нашу теорію, про те, що ми можемо відхилити Н0

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [107]:
from scipy.stats import chi2_contingency

In [113]:
contingency_table = pd.crosstab(df['version'], df['retention_7'])

chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

print(f'p-значення: {p_value:.3f}')

alpha = 0.05 
if p_value < alpha:
    print("Відхиляємо нульову гіпотезу: є залежність між версією гри та значенням retention_7.")
else:
    print("Не відхиляємо нульову гіпотезу: значення retention_7 не залежить від версії гри.")

p-значення: 0.002
Відхиляємо нульову гіпотезу: є залежність між версією гри та значенням retention_7.


Відхилення нульової гіпотези означає, що є статистично значуща залежність між версією гри та утриманням користувачів через 7 днів.

Отже, цей тест ще раз довів нам правдивість нашої гіпотези, яку ми зробила у попередньому тесті, що нульову гіпотезу си відкидаємо і є достатньо значущі показники, які свідчать, що інша версія гри (40) краще утримує гравців


